In [66]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn as sk
from sklearn.linear_model import BayesianRidge, LinearRegression, HuberRegressor, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import pickle as plk

data = pd.read_csv('no_pert.csv')
data.head()

,Unnamed: 0,ident,it,t,a(i),emegas(i),emepla(i)/emet,rplanet(i)/radtie,emestar,rc,qest,sigmag_0,emed,gama,apert,fpert,constmigI,emetal,taugas
0,11,1,199934,20000001.0,0.035424,37.450019,41.777421,2.500347,1.425386e+33,40.040195,2.281872,158.835559,0.18,1.0,0.0,1.0,0.1,-0.092791,3525593.5
1,12,1,199934,20000001.0,1.256601,0.994173,5.461369,2.529407,1.425386e+33,40.040195,2.281872,158.835559,0.18,1.0,0.0,1.0,0.1,-0.092791,3525593.5
2,13,1,199934,20000001.0,7.750644,0.000000,2.205831,2.001885,1.425386e+33,40.040195,2.281872,158.835559,0.18,1.0,0.0,1.0,0.1,-0.092791,3525593.5
3,14,1,199934,20000001.0,9.749447,0.000000,1.475556,1.751609,1.425386e+33,40.040195,2.281872,158.835559,0.18,1.0,0.0,1.0,0.1,-0.092791,3525593.5
4,15,1,199934,20000001.0,13.694444,0.000000,0.810833,1.435569,1.425386e+33,40.040195,2.281872,158.835559,0.18,1.0,0.0,1.0,0.1,-0.092791,3525593.5


In [11]:
#ident = tf.contrib.layers.sparse_column_with_keys(column_name="ident",keys=data.ident.unique())
#t = tf.contrib.layers.real_valued_column("t")
#a = tf.contrib.layers.real_valued_column("a(i)")
#emegas = tf.contrib.layers.real_valued_column("emegas(i)")
#emepla_emet = tf.contrib.layers.real_valued_column("emega(i)/emet")
#rplanet_radtie = tf.contrib.layers.real_valued_column("rplanet(i)/radtie")
#rc = tf.contrib.layers.real_valued_column("rc")
#sigmag_0 = tf.contrib.layers.real_valued_column("sigma_0")
#emed = tf.contrib.layers.real_valued_column("emed")
#apert = tf.contrib.layers.real_valued_column("apert")
#emetal = tf.contrib.layers.real_valued_column("emetal")
#taugas = tf.contrib.layers.real_valued_column("taugas")

In [16]:
Y = data[['a(i)','emegas(i)','emepla(i)/emet','rplanet(i)/radtie']]
X = data[['ident','emestar','rc','sigmag_0','emed','apert','emetal','taugas']]
Xsq = sk.preprocessing.PolynomialFeatures(degree=2)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [41]:
clf = BayesianRidge(compute_score=True,normalize=True)
clf.fit(X, Y['a(i)'])

ols = LinearRegression(normalize=True)
ols.fit(X, Y['a(i)'])

#hls = HuberRegressor()
#hls.fit(X, Y['a(i)'])
lls = Lasso(normalize=True)
lls.fit(X, Y['a(i)'])

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [38]:
y_pred = clf.predict(X)
sk.metrics.r2_score(Y['a(i)'], y_pred)

0.41114214929674131

In [39]:
y_pred = ols.predict(X)
sk.metrics.r2_score(Y['a(i)'], y_pred)

0.41114267608540378

In [42]:
y_pred = lls.predict(X)
sk.metrics.r2_score(Y['a(i)'], y_pred)

0.0

In [64]:
model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(normalize=True,n_jobs=-1))])
model.fit(X_train, Y_train['a(i)'])

Pipeline(steps=[('poly', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('linear', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True))])

In [65]:
y_pred = model.predict(X_test)
sk.metrics.r2_score(Y_test['a(i)'], y_pred)

0.4536832792820803

In [55]:
neural = MLPRegressor(learning_rate='adaptive',tol=1e-6,max_iter=400,hidden_layer_sizes=(400,))
neural.fit(X_train, Y_train['a(i)'])

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(400,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-06, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [56]:
y_pred = neural.predict(X_test)
sk.metrics.r2_score(Y_test['a(i)'], y_pred)

-3.4843070716552724e+54

In [67]:
mod = plk.load(open('model.p','rb'))

In [68]:
y_pred = mod.predict(X_test)
sk.metrics.r2_score(Y_test['a(i)'], y_pred)

0.4627556082548886